In [1]:
# import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path

In [2]:
# configuration
variable = 'rowe'
base_path = Path('Z:/nahaUsers/casadje/GloFASv4/long_run')
start_date = '1991-01-01'
end_date = '1992-12-31'
output_dir = base_path / variable / 'thresholds'
output_file = output_dir / f'{variable}_monthly_avg.nc'

# create output directory
output_dir.mkdir(parents=True, exist_ok=True)

# list of input NetCDF files
files = sorted((base_path / variable).glob(f'{variable}_*.nc'))
if not files:
    raise FileNotFoundError(f"No files found in {base_path / variable} matching pattern '{variable}_*.nc'.")

# open dataset and crop to the study period
ds = xr.open_mfdataset(
    files,
    engine='netcdf4',
    chunks='auto',
)[variable]

# rename variables
ds = ds.rename({'valid_time': 'time', 'latitude': 'lat', 'longitude': 'lon'})

# convert time convention to beginning-of-timestep
ds['time'] = ds['time'] - pd.Timedelta(days=1)

# crop to the study period
ds = ds.sel(time=slice(start_date, end_date))

In [4]:
# monthly resample
ds_monthly = ds.resample(time='1MS').mean(skipna=True)

In [5]:
# rechunk
ds_monthly = ds.chunk({'time': 1, 'lat':'auto', 'lon': 'auto'})

In [ ]:
# compute average
avg = ds_monthly.mean(['lat', 'lon'], skipna=True).compute()

In [ ]:
avg